In [1]:
#
#    Notebook de cours MAP412 - Chapitre 5 - M. Massot 2020-2021 - Ecole polytechnique
#    ----------   
#    Méthodes de résolution de systèmes linéaires itératives 
#    Poisson 1D, 2D, 3D
#    
#    Auteurs : L. Séries et M. Massot - (C) 2021
#    

In [2]:
import time
import numpy as np

from scipy.sparse import diags
from scipy.sparse.linalg import spsolve, cg

import matplotlib.pyplot as plt 

# Résolution de trois problèmes de Poisson 1D/2D/3D 

## Equation de Poisson

On souhaite résoudre le problème elliptique constitué par l'équation de Poisson soumise à des conditions aux limites de type de Dirichlet :

$$
\left\{
\begin{aligned}
-\Delta u & =  f & \text{dans} \; \Omega  \\
        u & =  g & \text{sur}  \;  \partial \Omega
\end{aligned}
\right.
$$

## Structure des matrices 1d, 2d et 3d pour une taille de matrice fixe

In [ ]:
nx = 125
dx = 1/(nx+1)
diag = np.repeat(2/dx**2, nx)
diag_x = np.repeat(-1/dx**2, nx-1)
a_1d  = diags([diag, diag_x, diag_x], [0, -1, 1])

nx = 11
ny = 11
dx = 1/(nx+1)
dy = 1/(ny+1)
diag = np.repeat(2/dx**2 + 2/dy**2, nx*ny)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny)
diag_y = np.repeat(-1/dy**2, nx*(ny-1))
a_2d = diags([diag, diag_x, diag_x, diag_y, diag_y], [0, -1, 1, -nx, nx])

nx = 5
ny = 5
nz = 5
dx = 1/(nx+1)
dy = 1/(ny+1)
dz = 1/(nz+1)
diag = np.repeat(2/dx**2 + 2/dx**2 + 2/dz**2, nx*ny*nz)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny*nz)
diag_y = np.tile(np.repeat([-1/dy**2, 0.], (nx*(ny-1), nz)), nz)
diag_z = np.repeat(-1/dz**2, nx*ny*(nz-1))
a_3d = diags([diag, diag_x, diag_x, diag_y, diag_y, diag_z, diag_z], [0, -1, 1, -nx, nx, -nx*ny, nx*ny])

plt.figure(figsize=[18,6])
ax1 = plt.subplot(1, 3, 1)
ax1.set_title('cas 1d')
plt.spy(a_1d, markersize=4)

ax2 = plt.subplot(1, 3, 2)
ax2.set_title('cas 2d')
plt.spy(a_2d, markersize=4) 

ax3 = plt.subplot(1, 3, 3)
ax3.set_title('cas 3d')
plt.spy(a_3d, markersize=4) 

plt.show()

## Valeurs propres du laplacien

In [ ]:
def eig_val_1d(nx, dx, i):
    a = (4/(dx**2))
    return a*np.sin((np.pi*i)/(2*(nx+1)))**2

def eig_val_2d(nx, dx, ny, dy, i, j):
    a = (4/(dx**2))
    b = (4/(dy**2))
    return a*np.sin((np.pi*i)/(2*(nx+1)))**2 + b*np.sin((np.pi*j)/(2*(ny+1)))**2

def eig_val_3d(nx, dx, ny, dy, nz, dz, i, j, k):
    a = (4/(dx**2))
    b = (4/(dy**2))
    c = (4/(dz**2))
    return a*np.sin((np.pi*i)/(2*(nx+1)))**2 + b*np.sin((np.pi*j)/(2*(ny+1)))**2 + c*np.sin((np.pi*k)/(2*(nz+1)))**2

In [ ]:
def conj_grad(a, b):

    nb_it = 0
    def callback(xk):
        nonlocal nb_it
        nb_it += 1

    t1 = time.time()
    x, info = cg(a, b, callback=callback)
    t2 = time.time()
    
    if info==0:
        print(f"Nb iteration : {nb_it}")
        print(f"Temps d'execution (s) : {t2-t1}")
    else:
        print(f"Convergence non atteinte en {info} itérations")

    return x

## Cas 1d

$$
\left\{
\begin{aligned}
- & u''(x)  =  f(x) \quad \text{dans} \; \Omega = [0,1] \quad \text{avec} \; f(x)=1\\
  & u(0)  =  0 \; \text{et} \; u(1) = 0
\end{aligned}
\right.
$$

In [ ]:
nx = 122500
nx = 122500
dx = 1/(nx+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2, nx)
diag_x = np.repeat(-1/dx**2, nx-1)
a  = diags([diag, diag_x, diag_x], [0, -1, 1])

# second membre
b = np.ones(nx)

print(f"Cas 1d : nx = {nx}")
print(f"Taille de la matrice : ({nx} x {nx})")
print(f"Largeur de bande : {3}")
print(f"Entrées de la matrice factorisée : {nx*3}")
print(f"Conditionnement : {eig_val_1d(nx, dx, nx)/eig_val_1d(nx, dx, 1)}")

print("\nRésolution par la méthode du gradient conjugué")
ucg = conj_grad(a, b)
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ucg))/np.linalg.norm(b)}")

t1 = time.time()    
ulu = spsolve(a.tocsr(), b)
t2 = time.time()
print("\nRésolution par une méthode directe")
print(f"Temps d'execution (s) : {t2-t1}")
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ulu))/np.linalg.norm(b)}")

## Cas 2d

$$
\left\{
\begin{aligned}
-\Delta u(x,y) & =  f(x,y) \quad \text{ dans } \; \Omega = [0,1] \times [0,1]  \quad \text{avec} \; f(x)=1 \\
        u(x,y) & =  0 \quad \text{ sur }  \;  \partial \Omega
\end{aligned}
\right.
$$

In [ ]:
nx = 350
ny = 350
dx = 1/(nx+1)
dy = 1/(ny+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2 + 2/dy**2, nx*ny)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny)
diag_y = np.repeat(-1/dy**2, nx*(ny-1))
a = diags([diag, diag_x, diag_x, diag_y, diag_y], [0, -1, 1, -nx, nx])

# second membre
b = np.ones(nx*ny)

print(f"Cas 2d : nx = {nx} et ny = {ny} => nx . ny = {nx*ny}")
print(f"Taille de la matrice : ({nx*ny} x {nx*ny})")
print(f"Largeur de bande : {2*nx}")
print(f"Entrées de la matrice factorisée : {nx*ny*(2*nx) }")
print(f"Conditionnement : {eig_val_2d(nx, dx, ny, dy, nx, ny)/eig_val_2d(nx, dx, ny, dy, 1, 1)}")

print("\nRésolution par la méthode du gradient conjugué")
ucg = conj_grad(a, b)
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ucg))/np.linalg.norm(b)}")

t1 = time.time()    
ulu = spsolve(a.tocsr(), b)
t2 = time.time()
print("\nRésolution par une méthode directe")
print(f"Temps d'execution : {t2-t1}")
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ulu))/np.linalg.norm(b)}")

## Cas 3d

$$
\left\{
\begin{aligned}
-\Delta u(x,y,z) & =  f(x,y,z) \quad \text{dans} \; \Omega = [0,1] \times [0,1] \times [0,1] \quad \text{avec} \; f(x)=1\\
        u(x,y,z) & =  0 \quad \text{sur}  \;  \partial \Omega
\end{aligned}
\right.
$$

In [ ]:
nx = 50
ny = 50
nz = 49
dx = 1/(nx+1)
dy = 1/(ny+1)
dz = 1/(nz+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2 + 2/dx**2 + 2/dz**2, nx*ny*nz)
diag_x = np.tile(np.repeat([-1/dx**2, 0.], (nx-1, 1)), ny*nz)
diag_y = np.tile(np.repeat([-1/dy**2, 0.], (nx*(ny-1), nz)), nz)
diag_z = np.repeat(-1/dz**2, nx*ny*(nz-1))
a = diags([diag, diag_x, diag_x, diag_y, diag_y, diag_z, diag_z], [0, -1, 1, -nx, nx, -nx*ny, nx*ny])

# second membre
b = np.ones(nx*ny*nz)

print(f"Cas 3d : nx = {nx}, ny = {ny} et nz = {nz} => nx . ny . nz = {nx*ny*nz}")
print(f"Taille de la matrice : ({nx*ny*nz} x {nx*ny*nz})")
print(f"Largeur de bande : {2*nx*ny}")
print(f"Entrées de la matrice factorisée : {nx*ny*nz*(2*nx*ny)}")
print(f"Conditionnement : {eig_val_3d(nx, dx, ny, dy, nz, dz, nx, ny, nz)/eig_val_3d(nx, dx, ny, dy, nz, dz, 1, 1, 1)}")

print("\nRésolution par la méthode du gradient conjugué")
ucg = conj_grad(a, b)
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ucg))/np.linalg.norm(b)}")

print("\nRésolution par une méthode directe")
t1 = time.time()    
ulu = spsolve(a.tocsr(), b)
t2 = time.time() 
print(f"Temps d'execution : {t2-t1}")
print(f"  ||A.xk - b|| / ||b|| = {np.linalg.norm(b - a.dot(ulu))/np.linalg.norm(b)}")